In [106]:
import pandas as pd
import numpy as np
import seaborn as sns
import tweepy
from tweepy import OAuthHandler
import yaml
import os
import json
import searchtweets
%matplotlib inline

In [107]:
with open('config.yaml', 'r') as file:
    twitter_auth = yaml.load(file)

In [108]:
consumer_key = twitter_auth['consumer_key']
consumer_secret= twitter_auth['consumer_secret']
access_token = twitter_auth['access_token']
access_token_secret = twitter_auth['access_token_secret']

In [109]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
 
api = tweepy.API(auth)

In [110]:
premium_search_args = searchtweets.load_credentials(".twitter_keys.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)

Grabbing bearer token from OAUTH


In [82]:
dates = ["2018-01-31", "2018-02-28", "2015-04-30", "2015-05-31"],
#          "2016-01-31", "2016-02-28", "2016-04-30", "2016-05-31", 
#          "2017-01-31", "2017-02-28", "2017-04-30", "2017-05-31"]

In [7]:
def retrieve_tweets(keyword="buhari", from_date="", to_date="", no_results=100):
    rule = searchtweets.gen_rule_payload(keyword,
                        from_date=from_date,
                        to_date=to_date,
                        results_per_call=no_results)
    rs = searchtweets.ResultStream(rule_payload=rule,
                  max_results=no_results,
                  max_pages=no_results,
                  **premium_search_args)
    tweets = list(rs.stream())
    return tweets

In [83]:
tweets = []
for date in dates:
    tweets += [*retrieve_tweets(from_date=date+" 12:00", to_date=date+" 17:00")]

In [84]:
len(tweets)

200

In [23]:
[print(tweet.all_text) for tweet in all_tweets[0:10]];

#samsonhorla Buhari meets Service Chiefs today over Boko Haram:  The fate of all the Service C... http://t.co/5p5tn4HTRn #teamfollowback
Nigeria: Buhari’s Biggest Challenge Is Tackling Corruption – Iko http://t.co/JnZWOMUC0J
Buhari meets Service Chiefs today over Boko Haram:  The fate of all the Service Chiefs will b... http://t.co/NWQSTeLmLp #Nigeria Comment
Buhari meets Service Chiefs today over Boko Haram:  The fate of all the Service Chiefs will be determined toda... http://t.co/6TKuAQyNiD
Dear Gov @elrufai
As President Buhari is heading to Chad, is he going to plan more BH attacks with Idris Deby?
Sincerely yours
Buhari/Osinbajo asset will be made public after due process as it will be checked in &amp; audit it value figures 1st https://t.co/ZcLSzMM1q0
GEJ fails to declare assets publicly #ProCorruption  GMB does same - CCB logic.  GEJ visits Chad on BH - Incompetent  GMB does same - Hero.
APC Senator-elect, Shehu Sani, Cautions Buhari on 'First Lady': http://t.co/uGyDTQtQk4 via @

In [24]:
all_tweets[0]

{'created_at': 'Mon Jun 01 23:58:59 +0000 2015',
 'id': 605523971967025152,
 'id_str': '605523971967025152',
 'text': '#samsonhorla Buhari meets Service Chiefs today over Boko Haram:  The fate of all the Service C... http://t.co/5p5tn4HTRn #teamfollowback',
 'source': '<a href="http://twitterfeed.com" rel="nofollow">twitterfeed</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 242893381,
  'id_str': '242893381',
  'name': 'I_AM_SUREGUY',
  'screen_name': 'samsonhorla',
  'location': 'Nigeria',
  'url': 'https://twitter.com',
  'description': 'I rep #teamGOD, #teamSUREGUY, #teamEconomics, #teamBarca #teamLOVINGGUY, #teamOOU, #teamFOLLOWBACK',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 1182,
  'friends_count': 760,
  'listed_count': 33,
  'favourites_count': 51,
  'statuses_count': 

In [33]:
tweets_df.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'matching_rules',
       'place', 'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'reply_count', 'retweet_count', 'retweeted', 'retweeted_status',
       'source', 'text', 'truncated', 'user'],
      dtype='object')

In [30]:
tweets_df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_id_str,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,None,None,Mon Jun 01 23:58:59 +0000 2015,NaN,"{'hashtags': [{'text': 'samsonhorla', 'indices...",NaN,NaN,0,False,low,...,NaN,NaN,0,1,False,NaN,"<a href=""http://twitterfeed.com"" rel=""nofollow...",#samsonhorla Buhari meets Service Chiefs today...,False,"{'id': 242893381, 'id_str': '242893381', 'name..."
1,None,None,Mon Jun 01 23:58:55 +0000 2015,NaN,"{'hashtags': [], 'urls': [{'url': 'http://t.co...",NaN,NaN,0,False,low,...,NaN,NaN,0,1,False,NaN,"<a href=""http://www.occuworld.org"" rel=""nofoll...",Nigeria: Buhari’s Biggest Challenge Is Tacklin...,False,"{'id': 1179710990, 'id_str': '1179710990', 'na..."
2,None,None,Mon Jun 01 23:58:55 +0000 2015,NaN,"{'hashtags': [{'text': 'Nigeria', 'indices': [...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,NaN,"<a href=""http://twitterfeed.com"" rel=""nofollow...",Buhari meets Service Chiefs today over Boko Ha...,False,"{'id': 914750738, 'id_str': '914750738', 'name..."
3,None,None,Mon Jun 01 23:58:55 +0000 2015,NaN,"{'hashtags': [], 'urls': [{'url': 'http://t.co...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,NaN,"<a href=""http://twitterfeed.com"" rel=""nofollow...",Buhari meets Service Chiefs today over Boko Ha...,False,"{'id': 971051394, 'id_str': '971051394', 'name..."
4,None,None,Mon Jun 01 23:58:53 +0000 2015,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,NaN,NaN,0,0,False,{'created_at': 'Mon Jun 01 22:18:43 +0000 2015...,"<a href=""http://twitter.com/download/iphone"" r...",RT @countfreemont: Dear Gov @elrufai\nAs Presi...,False,"{'id': 2992533929, 'id_str': '2992533929', 'na..."


In [85]:
tweets_df = pd.concat([tweets_df, pd.DataFrame(tweets)])

/Users/frederickodukomaiya/anaconda/envs/nlp/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [86]:
tweets_df.shape

(4300, 37)

In [87]:
tweets_df.created_at = pd.to_datetime(tweets_df.created_at)

In [62]:
tweets_df.to_csv("./data/buhari_tweets.csv")

In [88]:
tweets_df['created_date'] = tweets_df.created_at.apply(lambda x : x.strftime("%b %Y"))

In [89]:
tweets_df.sort_values(by=["created_at"], inplace=True)

In [91]:
tweets_df.reset_index(drop=True, inplace=True)

In [90]:
tweets_df.created_date.unique()

array(['Jan 2015', 'Feb 2015', 'Apr 2015', 'May 2015', 'Jun 2015',
       'Jul 2015', 'Aug 2015', 'Sep 2015', 'Oct 2015', 'Nov 2015',
       'Dec 2015', 'Jan 2016', 'Feb 2016', 'Mar 2016', 'Apr 2016',
       'May 2016', 'Jun 2016', 'Jul 2016', 'Aug 2016', 'Sep 2016',
       'Oct 2016', 'Nov 2016', 'Dec 2016', 'Jan 2017', 'Feb 2017',
       'Mar 2017', 'Apr 2017', 'May 2017', 'Jun 2017', 'Jul 2017',
       'Aug 2017', 'Sep 2017', 'Oct 2017', 'Nov 2017', 'Dec 2017',
       'Jan 2018', 'Feb 2018', 'Mar 2018', 'Jun 2018', 'Jul 2018',
       'Aug 2018', 'Sep 2018', 'Dec 2018'], dtype=object)

In [92]:
tweets_df.head()

,contributors,coordinates,created_at,created_date,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,...,quoted_status_id_str,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,None,None,2015-01-31 16:57:07,Jan 2015,NaN,"{'hashtags': [], 'urls': [{'url': 'http://t.co...",NaN,NaN,0,False,...,NaN,NaN,0,0,False,NaN,"<a href=""https://dlvrit.com/"" rel=""nofollow"">d...",I sacrificed my presidential ambition for Buha...,False,"{'id': 801409232, 'id_str': '801409232', 'name..."
1,None,None,2015-01-31 16:57:08,Jan 2015,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,...,NaN,NaN,0,4,False,NaN,"<a href=""http://www.twitter.com"" rel=""nofollow...",Buhari was stoned in Lagos -Obanikoro insists ...,False,"{'id': 42841145, 'id_str': '42841145', 'name':..."
2,None,None,2015-01-31 16:57:23,Jan 2015,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,...,NaN,NaN,0,0,False,{'created_at': 'Sat Jan 31 16:53:08 +0000 2015...,"<a href=""http://blackberry.com/twitter"" rel=""n...",RT @eXpozeMagazine: How can Buhari claim he is...,False,"{'id': 1924614565, 'id_str': '1924614565', 'na..."
3,None,None,2015-01-31 16:57:24,Jan 2015,NaN,"{'hashtags': [], 'urls': [{'url': 'http://t.co...",NaN,NaN,0,False,...,NaN,NaN,0,0,False,{'created_at': 'Fri Jan 30 16:15:15 +0000 2015...,"<a href=""https://twitter.com/download/android""...",RT @DailyPostNgr: Buhari is an illiterate – Ok...,False,"{'id': 497973086, 'id_str': '497973086', 'name..."
4,None,None,2015-01-31 16:57:26,Jan 2015,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,...,NaN,NaN,0,0,False,{'created_at': 'Sat Jan 31 15:55:23 +0000 2015...,"<a href=""http://www.twitter.com"" rel=""nofollow...",RT @SaharaReporters: Buhari’s top three platfo...,False,"{'id': 42841145, 'id_str': '42841145', 'name':..."


In [101]:
tweets_df.user[0]

{'id': 801409232,
 'id_str': '801409232',
 'name': 'Bidemi kosoko',
 'screen_name': 'bidemi_kosoko',
 'location': 'Lagos, Nigeria',
 'url': None,
 'description': 'best yoruba Nollywood act #ANTP , http://facebook.com/kosokobidemi',
 'translator_type': 'none',
 'protected': False,
 'verified': False,
 'followers_count': 4042,
 'friends_count': 1532,
 'listed_count': 22,
 'favourites_count': 5,
 'statuses_count': 63860,
 'created_at': 'Mon Sep 03 23:15:12 +0000 2012',
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'lang': 'en',
 'contributors_enabled': False,
 'is_translator': False,
 'profile_background_color': '352726',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme5/bg.gif',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme5/bg.gif',
 'profile_background_tile': False,
 'profile_link_color': 'D02B55',
 'profile_sidebar_border_color': '829D5E',
 'profile_sidebar_fill_color': '99CC33',
 'profile_text_color': '3

In [102]:
tweets_df['username'] = tweets_df.user.apply(lambda x: x['name'])
tweets_df['user_location'] = tweets_df.user.apply(lambda x: x['location'])

In [105]:
tweets_df.user_location.value_counts()

Nigeria                                     426
Lagos, Nigeria                              300
Abuja, Nigeria                              131
Lagos                                        88
Abuja                                        79
Lagos Nigeria                                33
Biafra                                       24
Kaduna, Nigeria                              21
Africa                                       21
Houston TX                                   21
Port Harcourt, Nigeria                       20
lagos                                        20
Germany                                      20
Worldwide                                    19
Nigeria                                      17
NIGERIA                                      16
Lagos, Nigeria.                              16
Kano, Nigeria                                16
İstanbul, Türkiye                            15
Tottenham, London                            15
United States                           